# Deploy an MLflow model with SageMaker

## Install MLflow

In [1]:
!pip install -q mlflow==2.0.1

In [2]:
!pip install scikit-learn==1.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 180.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


## Setup environment

In [3]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston
from mlflow.deployments import get_deploy_client

# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = sagemaker.get_execution_role() 
# uri of your remote mlflow server
tracking_uri = 'http://MLflow-MLFLO-VR3wJBi9L1nl-09870ac901267fb3.elb.us-east-1.amazonaws.com/' 
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/29/25 09:22:44] INFO     Found credentials from IAM Role:                                   ]8;id=580159;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=348825;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[03/29/25 09:22:54] INFO     Found credentials from IAM Role:                                   ]8;id=584226;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=425694;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=653272;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=876801;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## Build MLflow docker image to serve the model with SageMaker 

### Instructions to Build Your Own MLflow Docker Image to Serve a Model

You can do this on your local Docker daemon.

1. Make sure you have configured AWS credentials on your PC:

    ```sh
    $ aws configure
    AWS Access Key ID [****]: Your-Access-Key-ID
    AWS Secret Access Key [****]: Your-Secret-Access-Key
    Default region name [None]: us-east-1
    Default output format [None]: json
    ```

2. Install Docker Desktop https://www.docker.com/products/docker-desktop/ (ensure your Docker Desktop is up and running before proceeding with the following steps).

3. Install MLflow:

    ```sh
    $ pip install mlflow
    ```

4. Install Boto3:

    ```sh
    $ pip install boto3
    ```

5. Run the following command to build and push the MLflow container. This step will take 20 to 30 minutes to complete and will automatically push the container to Amazon ECR:

    ```sh
    $ mlflow sagemaker build-and-push-container
    ```

Once you have your own Docker image, you can start using it.


In [ ]:
#!aws ecr get-login --no-include-email --registry-ids 146171926552 --region us-east-1

In [ ]:
##MLFlow Docker Setup

In [4]:
!docker images

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


In [5]:
!docker pull 03sarath/mlflow-pyfunc:2.12.2 #Pull Docker images from my registry

2.12.2: Pulling from 03sarath/mlflow-pyfunc

c94e5e10: Pulling fs layer 
1a841b62: Pulling fs layer 
92926c51: Pulling fs layer 
4141f19b: Pulling fs layer 
d50a325e: Pulling fs layer 
2a40ba8d: Pulling fs layer 
e46d18db: Pulling fs layer 
75548dfe: Pulling fs layer 
b23d2876: Pulling fs layer 
755c6ff6: Pulling fs layer 
9c3cf88b: Pulling fs layer 
22d7abdc: Pulling fs layer 
6493764b: Pulling fs layer 
6e1a3d00: Pulling fs layer 
8e2e887d: Pulling fs layer 
Digest: sha256:e14fd53a9ff37d91cbf775c02749bdc16cff931e482cfbabd4e0f904f85ce41f
Status: Downloaded newer image for 03sarath/mlflow-pyfunc:2.12.2
docker.io/03sarath/mlflow-pyfunc:2.12.2


In [6]:
!docker images

REPOSITORY               TAG       IMAGE ID       CREATED         SIZE
03sarath/mlflow-pyfunc   2.12.2    bef1a9579b21   10 months ago   2.49GB


In [7]:
#!docker tag 03sarath/mlflow-pyfunc:2.12.2 <You'r ECR repo ID>
!docker tag 03sarath/mlflow-pyfunc:2.12.2 010526276239.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2

In [8]:
!docker images

REPOSITORY                                            TAG       IMAGE ID       CREATED         SIZE
010526276239.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.12.2    bef1a9579b21   10 months ago   2.49GB
03sarath/mlflow-pyfunc                                2.12.2    bef1a9579b21   10 months ago   2.49GB


In [9]:

#!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <URI OF YOUR REPO>
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 010526276239.dkr.ecr.us-east-1.amazonaws.com/mlflow

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [11]:
#!docker push <You'r ECR repo ID>
!docker push 010526276239.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2

The push refers to repository [010526276239.dkr.ecr.us-east-1.amazonaws.com/mlflow]

1a00fa36: Preparing 
287d86ab: Preparing 
b1f69ad5: Preparing 
d0fff421: Preparing 
631509cb: Preparing 
4ef5159e: Preparing 
d9d5154a: Preparing 
b34dda3e: Preparing 
30edc673: Preparing 
f42004d5: Preparing 
d5bd131d: Preparing 
d78ffadd: Preparing 
11ab8595: Preparing 
164da43b: Preparing 
8d0ece3b: Preparing 


b34dda3e: Pushing  554.9MB/780.5MBPushing  83.43MB/288.5MBPushing  67.95MB/72.81MB

164da43b: Pushed   711.4MB/704.9MB2.12.2: digest: sha256:14ff02ed7878c0ebf82840a2557b28ba0c56b44e7a6fbfb425a555aa08a0efd8 size: 3689


In [12]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '010526276239.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2'

## Deploy a SageMaker endpoint with our scikit-learn model

In [26]:
endpoint_name = 'boston-housing-mlops'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/boston/2'

In [27]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region
}

client = get_deploy_client("sagemaker")

# client.create_deployment(
#     name=endpoint_name,
#     model_uri=model_uri,
#     flavor='python_function',
#     config=config
# )


# Use update_deployment to update the existing endpoint with the new model version.
client.update_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

2025/03/29 10:25:38 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2025/03/29 10:25:38 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket


[03/29/25 10:25:38] INFO     Found credentials from IAM Role:                                   ]8;id=27209;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=421035;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/29/25 10:25:39] INFO     Found credentials from IAM Role:                                   ]8;id=444894;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=476207;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

2025/03/29 10:25:39 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-1-010526276239` already exists. Skipping creation.


                    INFO     Found credentials from IAM Role:                                   ]8;id=26467;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=634676;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

2025/03/29 10:25:39 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': '5Z2W9V8WCXY40ZT7', 'HostId': 'M/NyFF5lmHQP2F8ztScMBhdSKWc0AmilVeAeLDwLiCGkaithioZvE733PkF+kvuQUul4124cJNs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'M/NyFF5lmHQP2F8ztScMBhdSKWc0AmilVeAeLDwLiCGkaithioZvE733PkF+kvuQUul4124cJNs=', 'x-amz-request-id': '5Z2W9V8WCXY40ZT7', 'date': 'Sat, 29 Mar 2025 10:25:40 GMT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
2025/03/29 10:25:40 INFO mlflow.sagemaker: Found active endpoint with arn: arn:aws:sagemaker:us-east-1:010526276239:endpoint/boston-housing-mlops. Updating...
2025/03/29 10:25:40 INFO mlflow.sagemaker: Created new model with arn: arn:aws:sagemaker:us-east-1:010526276239:model/boston-housing-mlops-model-6t7meqhureagqmsy8rmndq
2025/03/29 10:25:41 INFO mlflow.sagemaker: Created new endpoint configuration with arn: arn:aws:sagemaker:us-east-1:010526276239:endpoint-config/boston-housing-mlops-config-86pzvts8sbmlxqyva

{'name': 'boston-housing-mlops', 'flavor': 'python_function'}

## Predict

In [28]:
# load boston dataset
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [29]:
client = get_deploy_client(f"sagemaker:/{region}")

payload = df.iloc[[5]]
prediction = client.predict(endpoint_name, df.iloc[[5]])

print(f'Payload: {payload}')
print(f'Prediction: {prediction}')

Payload:       CRIM   ZN  INDUS  CHAS    NOX    RM   AGE     DIS  RAD    TAX  PTRATIO  \
5  0.02985  0.0   2.18   0.0  0.458  6.43  58.7  6.0622  3.0  222.0     18.7   

        B  LSTAT  
5  394.12   5.21  
Prediction: {'predictions': [26.32441503321641]}


## Delete endpoint

In [30]:
client.delete_deployment('boston-housing-mlops', config=config)

2025/03/29 10:32:47 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:us-east-1:010526276239:endpoint/boston-housing-mlops
2025/03/29 10:32:47 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2025/03/29 10:32:47 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2025/03/29 10:32:52 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2025/03/29 10:32:52 INFO mlflow.sagemaker: Cleaning up unused resources...
2025/03/29 10:32:52 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:us-east-1:010526276239:endpoint-config/boston-housing-mlops-config-86pzvts8sbmlxqyvar-ghq
2025/03/29 10:32:53 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:us-east-1:010526276239:model/boston-housing-mlops-model-6t7meqhureagqmsy8rmndq
